In [6]:
import codecs

# Change fp to tweets csv of social_spambots_2.csv from http://mib.projects.iit.cnr.it/dataset.html
bots_fp = '/home/david/Desktop/social_spambots_2.csv/tweets.csv'
genuine_fp = '/home/david/Desktop/genuine_accounts.csv/tweets.csv'

# Open csv file and get the tweet part of the csv.
# Strip out newlines and quotes around text.
with codecs.open(bots_fp, 'r', encoding='utf-8', errors='ignore') as bots_file:
    bot_sentences = [x.split(',')[1].strip('\n').strip('"').lower()
                 if len(x.split(',')) > 1 else '' for x in bots_file.readlines()]
bot_sentences = bot_sentences[1:]
with codecs.open(genuine_fp, 'r', encoding='utf-8', errors='ignore') as genuine_file:
    genuine_sentences = [x.split(',')[1].strip('\n').strip('"').lower()
                 if len(x.split(',')) > 1 else '' for x in genuine_file.readlines()]
genuine_sentences = genuine_sentences[1:]

In [7]:
# Tokenize sentences for gensim word2vec.
tokenizes_sentences = [sentence.split(' ') for sentence in genuine_sentences]    

In [8]:
print(len(tokenizes_sentences))
print(tokenizes_sentences[0])

2860295
['rt', '@morningjewshow:', 'speaking', 'about', 'jews', 'and', 'comedy', 'tonight', 'at', 'temple', 'emanu-el', 'in', 'san', 'francisco.', 'in', 'other', 'words']


In [9]:
from gensim.models import Word2Vec
# Train a word2vec model.
model = Word2Vec(tokenizes_sentences, min_count=1)

In [14]:
list(model.wv.vocab)[:10]

['rt',
 '@morningjewshow:',
 'speaking',
 'about',
 'jews',
 'and',
 'comedy',
 'tonight',
 'at',
 'temple']

In [11]:
model['dad']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-3.5086179e+00, -6.4968365e-01, -2.3148289e+00,  1.0698880e-01,
       -7.3977137e-01, -1.4446214e-01,  2.9432261e+00,  9.9253863e-01,
       -3.4304583e-01,  5.0636637e-01, -3.0197265e+00,  8.2303566e-01,
        2.9956572e+00, -5.1384866e-01, -8.4286642e-01, -3.8677725e-05,
        2.2685297e+00,  2.5673730e+00, -5.1027727e-01,  8.0755931e-01,
        7.7935171e-01,  3.2813114e-01,  1.3714983e+00,  4.6107167e-01,
       -7.2718924e-01, -8.3830500e-01,  5.2966356e-01,  1.6572822e+00,
       -4.6613817e+00,  1.9734647e+00,  1.5797269e+00,  1.2802039e+00,
       -1.4272243e+00,  1.1769537e-01,  2.7637258e+00, -1.0508411e+00,
        4.9056172e-01, -1.7329830e+00,  2.5264430e+00,  1.6759747e+00,
        6.6929770e-01, -8.0651945e-01,  1.4680361e+00, -3.0158200e+00,
        9.4583440e-01, -1.6171439e+00, -1.9272529e+00,  1.2858894e+00,
       -2.4988508e+00,  2.2832789e+00,  2.5144823e+00,  2.0674613e+00,
       -3.0779338e+00,  9.1380161e-01,  2.0189592e-01,  6.4543658e-01,
      

In [12]:
model.most_similar(positive=['dad'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('mum', 0.9454367160797119),
 ('mom', 0.9385337829589844),
 ('sister', 0.9177341461181641),
 ('grandma', 0.9141216278076172),
 ('brother', 0.881198525428772),
 ('daughter', 0.8643038272857666),
 ('husband', 0.8602260947227478),
 ('wife', 0.8548371195793152),
 ('cousin', 0.8535829782485962),
 ('nephew', 0.8311100602149963)]

In [15]:
lens = [len(x) for x in tokenizes_sentences]

In [22]:
import statistics
print('mean', statistics.mean(lens))
print('median', statistics.median(lens))
print('max', max(lens))
# Tokenize sentences for gensim word2vec.
bot_tokenizes_sentences = [sentence.split(' ') for sentence in bot_sentences]
bot_lens = [len(x) for x in bot_tokenizes_sentences]
print('mean', statistics.mean(bot_lens))
print('median', statistics.median(bot_lens))
print('max', max(bot_lens))

mean 9.737832985758462
median 8
max 131
mean 9.880279685691889
median 9.0
max 78


In [26]:
# Tokenize sentences for gensim word2vec.
genuine_tokenizes_sentences = [sentence.split(' ') for sentence in genuine_sentences]
bot_tokenizes_sentences_10 = list(filter(lambda x: len(x) <= 10, bot_tokenizes_sentences))
genuine_tokenizes_sentences_10 = list(filter(lambda x: len(x) <= 10, genuine_tokenizes_sentences))

In [28]:
len(genuine_tokenizes_sentences_10)

1771034

In [29]:
# Following https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
def create_training_data(tokenized_sentences, word2vec_model, pos_or_neg):
    '''
    Creates training data by returing the embedded version of the sentence with the label.
    Currently only using tweets of 10 words or less.
    
    :tokenized_sentences: list of list of words
    :word2vec_model: trained word2vec model
    :pos_or_neg: 0 if genuine, 1 if bot
    
    Returns:
    X: list of embedded sentences
    Y: List of Labels
    '''
    # TODO: Prune tokenized_sentences that do not have words in the word2vec_model
    pruned_tokenized_sentences = []
    # TODO: Create embedding via word2vec model and pad 0 vectors for sentences less than 10

    X = []
    Y = [pos_or_neg] * len(pruned_tokenized_sentences)
    return X, Y